
# Preprocessing & Training

## NeuroSense Analytics 

### v0.1.5

#### Planned Features  

- Dedicated preprocessor class for LSTM model
- LSTM model class for training
- Pipelining preprocessing into training
- Hyperparameter optimisation using Optuna


# Documentation  

## DataPreprocessor  

Creating a `preprocessor` class to automate the import workflow - preprocessor handles data import, datatype casting, one-hot encoding, MICE imputation, joining feature data with survey data and returns a Polars dataframe or NumPy series.  

Output is in the shape of `batch_size`, `time_steps`, `features` for LSTM processing.   

> ### Parameters:  

- `path`: *`{1, 2, 3, 4}`*  
Data path to use (INS_W1 = `1`, INS_W2 = `2`, INS_W3 = `3`, INS_W4 = `4`)  

- `imputer_max_iter`: *`int`, default = `10`*  
Max amount of iterations for IterativeImputer.  

- `imputer_random_state`: *`int`, default = `42`*  
Imputer random state.  

- `nearest_features`: *`int`, default = `None`*  
How many neighbours to sample when imputing.  

- `strategy`: *`{‘mean’, ‘median’, ‘most_frequent’, ‘constant’}`, default = `median`*  
What strategy to use when imputing  

- `impute`: *`bool`, default = `True`*  
Whether or not to run imputation.  

- `exclude_history`: *`bool`, default = `True`*  
Whether or not to exclude 14- and 7-day histories during preprocessing.

> ### Functions:  

- `import_csv_feature_data(csv)`:  
Imports and normalises feature dataset. Returns `pl.DataFrame`.

    > Parameters:  

    - `csv`: *`str`*   
    File name of csv to be processed.  

- `import_csv_survey_data(csv)`:  
Imports and normalises survey dataset. Returns `pl.DataFrame`. 

    > Parameters:  

    - `csv`: *`str`*  
    File name of csv to be processed.  
    
- `import_dep_endterm()`:  
Imports, one-hot encodes and normalises the `dep_endterm` dataset for the selected datapath. Returns `pl.DataFrame`.

- `merge_dataframe(dataframe_1, dataframe_2, join_type)`:  
Merges two dataframes on `join_type`, returns `pl.DataFrame`.  

    > Parameters:  

    - `dataframe_1`: *`pl.DataFrame`*  
    DataFrame to join with. 

    - `dataframe-2`: *`pl.DataFrame`*  
    DataFrame to be joined to `dataframe_1`.  

    - `join_type`: *`{‘inner’, ‘left’, ‘right’, ‘full’, ‘semi’, ‘anti’, ‘cross’}, default="inner"`*  
    Join strategy.
    

> ### Example Usage:   

`preprocessor_INS_W1 = DataPreprocessor(1, imputer_max_iter=20)`


## PreprocessorLSTM

## ModelTraining

# Imports

In [2]:
import numpy as np
import polars as pl
from datetime import date
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from tensorflow.keras.utils import pad_sequences
import logging

# Class Declaration  

## DataPreprocessor

In [3]:
# This script imports CSV files containing feature and survey data, processes them, and prepares them for analysis.
# It also includes functions to load and preprocess the data, including scaling and encoding categorical variables.

# data path for the CSV files
# The data is organized into four directories, each containing feature and survey data.

class DataPreprocessor:
    def __init__(self, path: str, imputer_max_iter: int = 10, imputer_random_state: int = 42, nearest_features: int = None, strategy: str = "mean", only_history: bool = True, impute: bool = True): # Initialize the DataPreprocessor class
        self.only_history = only_history # Include only history data if specified
        self.scaler = MinMaxScaler() # Initialize the MinMaxScaler with specified parameters
        self.impute = impute
        self.imputer = IterativeImputer(max_iter=imputer_max_iter, random_state=imputer_random_state, n_nearest_features=nearest_features, initial_strategy=strategy) # Initialize the IterativeImputer with specified parameters

        match path: ## Set the path based on the input parameter
            case 1:
                self.path = "./csv_data/INS-W_1/"
            case 2:
                self.path = "./csv_data/INS-W_2/"
            case 3:
                self.path = "./csv_data/INS-W_3/"
            case 4:
                self.path = "./csv_data/INS-W_4/"
            case _:
                raise ValueError("Invalid path specified. Please choose a valid path (1, 2, 3, or 4).")

    # Load the CSV files, cast columns to appropriate types, and drop empty columns
    def import_csv_feature_data(self, file_name: str) -> pl.DataFrame:
        try:
            q = (
                pl.scan_csv(self.path + "FeatureData/" + file_name + ".csv")
                .select(pl.col("*"))
                .cast({"date": pl.Date})
                .drop("")
                .with_columns(pl.col("pid").str.replace_all("INS-W_",""))
                .cast({"pid": pl.Int32})
                .select(pl.exclude(pl.String))
            )
            data = q.collect() # Collect the lazy frame into a DataFrame

            if self.impute:
                if self.only_history: # If only history data is to be included
                    data = data.select(pl.col(["pid","date","^.*14dhist$","^.*7dhist$"]))
                scaled_data = pl.from_numpy( # Convert to numpy array for scaling
                        self.scaler.fit_transform(data.select(pl.exclude([pl.Date, pl.Int32]))), schema=data.select(pl.exclude([pl.Date, pl.Int32])).columns # min max scaling on all columns except date and pid
                    )
                try:
                    self.imputer.fit(scaled_data) # Fit the imputer to the scaled data
                    imputed_data = pl.from_numpy(
                        self.imputer.transform(scaled_data), schema=data.select(pl.exclude(["pid","date"])).columns # Transform the scaled data using the imputer
                    )
                    data = data.select(["pid","date"])
                    data = data.hstack(imputed_data) # Add imputed data back to the DataFrame
                    del imputed_data # Delete the imputed data variable to free up memory
                except: 
                    print("Error in imputation, returning scaled data without imputation.")
                    return data
                return data
            return data
        
        except Exception as e:
            print(f"Error importing feature data from {self.path + 'FeatureData/' + file_name}: {e}")
            return pl.DataFrame()

    def import_csv_survey_data(self, file_name: str) -> pl.DataFrame:
        try: # Load survey data from CSV file
            q = (
                pl.scan_csv(self.path + "SurveyData/" + file_name + ".csv")
                .select(pl.col("*"))
                .cast({"date": pl.Date})
                .drop("")
                .with_columns(pl.col("pid").str.replace_all("INS-W_",""))
                .cast({"pid": pl.Int32})
            )
            data = q.collect()
            match file_name:
                case "ema":
                    survey_data = data.select(pl.exclude(["pid","date"])) # Convert to numpy array for scaling
                    scaled_data = pl.from_numpy(
                        self.scaler.fit_transform(survey_data), schema=survey_data.columns
                    )
                    data = data.select(["pid","date"])
                    data = data.hstack(scaled_data)
                    del scaled_data
                    return data
                case "post":
                    survey_data = data.select(pl.exclude(["pid","date"])) # Convert to numpy array for scaling
                    scaled_data = pl.from_numpy(
                        self.scaler.fit_transform(survey_data), schema=survey_data.columns
                    )
                    data = data.select(["pid","date"])
                    data = data.hstack(scaled_data)
                    del scaled_data
                    return data
                case "pre":
                    survey_data = data.select(pl.exclude(["pid","date"]))
                    scaled_data = pl.from_numpy(
                        self.scaler.fit_transform(survey_data), schema=survey_data.columns
                    )
                    data = data.select(["pid","date"])
                    data = data.hstack(scaled_data)
                    del scaled_data
                    return data
        except Exception as e:
            print(f"Error importing survey data from {self.path + 'SurveyData/' + file_name}: {e}")
            return pl.DataFrame()

    def import_dep_endterm(self) -> pl.DataFrame:
        try:
            q = (
                pl.scan_csv(self.path + "SurveyData/dep_endterm.csv")
                .select(pl.col("*"))
                .cast({"date": pl.Date})
                .drop("")
                .with_columns(pl.col("pid").str.replace_all("INS-W_",""))
                .cast({"pid": pl.Int32})    
                )
            data = q.collect()
            bdi2 = data.select(pl.exclude(["pid","date", "dep"]))
            data_scaled = pl.from_numpy(
                self.scaler.fit_transform(bdi2), schema=bdi2.columns # min max scaling on all columns except date and pid
            )
            data = data.select(["pid","date", "dep"])
            data = data.hstack(data_scaled) # Add scaled data back to the DataFrame
            del data_scaled # Delete the scaled data variable to free up memory
            data = data.to_dummies("dep")
            return data
        except Exception as e:
            print(f"Error importing endterm data from {self.path + 'SurveyData/dep_endterm.csv'}: {e}")
            return pl.DataFrame()

    def merge_dataframe(self, dataframe_1: pl.DataFrame, dataframe_2: pl.DataFrame, join_type: str = "inner") -> pl.DataFrame:
        try:
            merged_data = dataframe_1.join(dataframe_2, on=["pid", "date"], how=join_type) # Merge feature and survey data on 'pid'
            return merged_data
        except Exception as e:
            print(f"Error merging feature and survey data: {e}")
            return pl.DataFrame()
    
    def import_dep_weekly(self) -> pl.DataFrame:
        try:
            q = (
                pl.scan_csv(self.path + "SurveyData/dep_weekly.csv")
                .select(pl.col("*"))
                .cast({"date": pl.Date})
                .drop("")
                .with_columns(pl.col("pid").str.replace_all("INS-W_",""))
                .cast({"pid": pl.Int32})
                .select(pl.col(["pid","date","dep_weeklysubscale_endterm_merged"]))
                )
            data = q.collect()
            data = data.to_dummies("dep_weeklysubscale_endterm_merged") # Convert categorical variable to dummy variables
            return data
        
        except Exception as e:
            print(f"Error importing weekly data from {self.path + 'SurveyData/dep_weekly.csv'}: {e}")
            return pl.DataFrame()
        
    def merge_on_date(self, dataframe_1: pl.DataFrame, dataframe_2: pl.DataFrame) -> pl.DataFrame:
        try:
            merged_data = dataframe_1.join(dataframe_2, on=["date"], how="inner") # Merge feature and survey data on 'date'
            return merged_data
        except Exception as e:
            print(f"Error merging dataframes on date: {e}")
            return pl.DataFrame()

## ModelLSTM

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking

class ModelLSTM:
    def __init__(self): # Initialize the PreprocessorLSTM class
        self.logger = logging.getLogger(__name__) # Initialize the logger for the PreprocessorLSTM class
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s') # Set the logging level and format
        self.logger.info("PreprocessorLSTM initialized") # Log the initialization of the PreprocessorLSTM class
        self.padded_sequences = None # Initialize the padded sequences variable

        
    def select_features(self, data: pl.DataFrame, features: list) -> pl.DataFrame:
        try:
            selected_data = data.select(pl.col(features)) # Select specified features from the DataFrame
            self.logger.info(f"Selected features: {features}")
            return selected_data
        except Exception as e:
            self.logger.error(f"Error selecting features: {e}")
            return pl.DataFrame()


    def create_padded_sequences(self, data: pl.DataFrame) -> np.ndarray:
        self.sequences = data.rows()
        try:
            self.padded_sequences = pad_sequences(
                self.sequences, padding='post', dtype='float32'
            )
            self.logger.info(f"Created padded sequences with shape: {self.padded_sequences.shape}")
            return self.padded_sequences
        
        except Exception as e:
            self.logger.error(f"Error creating padded sequences: {e}")
            return np.array([])
    

    def split_data(self, padded_sequences: np.ndarray, test_size: float = 0.2) -> tuple:
        try:
            self.participant_ids = np.arange(len(padded_sequences))  # [0, 1, 2, ...]
            self.train_ids, test_ids = train_test_split(self.participant_ids, test_size=test_size, shuffle=False) # Split the data into training and testing sets
            self.X_train = padded_sequences[self.train_ids]
            self.X_test = padded_sequences[test_ids]
            self.y_train = np.array([seq[1:, :] for seq in self.X_train])  # Shift by 1 time step
            self.y_test = np.array([seq[1:, :] for seq in self.X_test])
            # Logging
            self.logger.info(f"Split data into train and test sets with sizes: {len(self.X_train)}, {len(self.X_test)}")
            self.logger.info(f"Train and test data shapes: {self.X_train.shape}, {self.X_test.shape}")
            self.logger.info(f"Train and test labels shapes: {self.y_train.shape}, {self.y_test.shape}")
            return self.X_train, self.X_test, self.y_train, self.y_test
        
        except Exception as e:
            self.logger.error(f"Error splitting data: {e}")
            return None, None, None, None
    
    
    def build_lstm_model(self, n_features):
        try:
            inputs = Input(shape=(None, n_features))  # Variable-length sequences
            x = Masking(mask_value=0.0)(inputs)      # Ignore padded zeros
            x = LSTM(64, return_sequences=True)(x)  # Process entire sequence
            outputs = Dense(n_features)(x)           # Predict all features
            model = Model(inputs, outputs)
            model.compile(optimizer="adam", loss="mse")
            self.logger.info("LSTM model built successfully")
            self.logger.info(f"Model summary: {model.summary()}")
            return model
        
        except Exception as e:
            self.logger.error(f"Error building LSTM model: {e}")
            return None
            

In [ ]:
TestModel = ModelLSTM() # Create an instance of the ModelLSTM class
model = TestModel.build_lstm_model(n_features=2)
model.fit(TestModel.X_train, TestModel.y_train, epochs=10, validation_data=(TestModel.X_test, TestModel.y_test)) # Since x_train and y_train are initialized in the class we can use them directly

# INS-W_1

In [47]:
PreprocessorLSTM = PreprocessorLSTM()

2025-05-12 10:47:43,105 - INFO - PreprocessorLSTM initialized


In [6]:
preprocessor_INS_W1 = DataPreprocessor(1, imputer_max_iter=10, nearest_features=10)

# Testing, ignore this section

In [7]:
rapids_1 = preprocessor_INS_W1.import_csv_feature_data("rapids")

c:\Users\darkenral\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [8]:
dep_weekly_1 = preprocessor_INS_W1.import_dep_weekly()

In [6]:
dep_weekly_1.sample(10)

pid,date,dep_weeklysubscale_endterm_merged_false,dep_weeklysubscale_endterm_merged_true
i32,date,u8,u8
20,2018-05-09,1,0
198,2018-04-18,1,0
168,2018-05-13,1,0
163,2018-05-20,1,0
44,2018-05-02,1,0
121,2018-05-13,1,0
177,2018-05-30,1,0
13,2018-04-18,1,0
38,2018-04-11,0,1


In [9]:
rapids_weekly = dep_weekly_1.join(rapids_1, on=["pid","date"], how="left")

In [31]:
rapids_weekly_float = rapids_weekly.select(pl.exclude(pl.Date))

In [32]:
rapids_weekly_float.head()

pid,dep_weeklysubscale_endterm_merged_false,dep_weeklysubscale_endterm_merged_true,f_slp:fitbit_sleep_summary_rapids_sumdurationafterwakeupmain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationtofallasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationinbedmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationafterwakeupmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationawakemain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationtofallasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationinbedmain:14dhist,f_slp:fitbit_sleep_summary_rapids_countepisodemain:14dhist,f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:14dhist,f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:14dhist,f_slp:fitbit_sleep_summary_rapids_firstwaketimemain:14dhist,f_slp:fitbit_sleep_summary_rapids_lastwaketimemain:14dhist,f_slp:fitbit_sleep_intraday_rapids_avgdurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_avgdurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_maxdurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_maxdurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_sumdurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_sumdurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_countepisodeasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_countepisodeawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_stddurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_stddurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mindurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mindurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mediandurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mediandurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_ratiocountasleepunifiedwithinmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_ratiocountawakeunifiedwithinmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_ratiodurationawakeunifiedwithinmain:14dhist,…,f_loc:phone_locations_barnett_probpause_norm:7dhist,f_loc:phone_locations_barnett_rog_norm:7dhist,f_loc:phone_locations_barnett_siglocentropy_norm:7dhist,f_loc:phone_locations_barnett_siglocsvisited_norm:7dhist,f_loc:phone_locations_barnett_stdflightdur_norm:7dhist,f_loc:phone_locations_barnett_stdflightlen_norm:7dhist,f_loc:phone_locations_barnett_wkenddayrtn_norm:7dhist,f_loc:phone_locations_doryab_avglengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_avgspeed_norm:7dhist,f_loc:phone_locations_doryab_homelabel_norm:7dhist,f_loc:phone_locations_doryab_locationentropy_norm:7dhist,f_loc:phone_locations_doryab_locationvariance_norm:7dhist,f_loc:phone_locations_doryab_loglocationvariance_norm:7dhist,f_loc:phone_locations_doryab_maxlengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_minlengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_movingtostaticratio_norm:7dhist,f_loc:phone_locations_doryab_normalizedlocationentropy_norm:7dhist,f_loc:phone_locations_doryab_numberlocationtransitions_norm:7dhist,f_loc:phone_locations_doryab_numberofsignificantplaces_norm:7dhist,f_loc:phone_locations_doryab_outlierstimepercent_norm:7dhist,f_loc:phone_locations_doryab_radiusgyration_norm:7dhist,f_loc:phone_locations_doryab_stdlengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_timeathome_norm:7dhist,f_loc:phone_locations_doryab_timeattop1location_norm:7dhist,f_loc:phone_locations_doryab_timeattop2location_norm:7dhist,f_loc:phone_locations_doryab_timeattop3location_norm:7dhist,f_loc:phone_locations_doryab_totaldistance_norm:7dhist,f_l

In [48]:
rapids_padded = PreprocessorLSTM.create_padded_sequences(rapids_weekly_float)

2025-05-12 10:47:48,102 - INFO - Created padded sequences with shape: (2360, 819)


In [53]:
print(np.arange(len(rapids_padded)).shape)

(2360,)


In [49]:
x_train, x_test, y_train, y_test = PreprocessorLSTM.split_data(rapids_padded)

2025-05-12 10:47:54,370 - ERROR - Error splitting data: too many indices for array: array is 1-dimensional, but 2 were indexed


In [11]:
rapids_weekly.head(10)

pid,date,dep_weeklysubscale_endterm_merged_false,dep_weeklysubscale_endterm_merged_true,f_slp:fitbit_sleep_summary_rapids_sumdurationafterwakeupmain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationtofallasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_sumdurationinbedmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationafterwakeupmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationawakemain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationtofallasleepmain:14dhist,f_slp:fitbit_sleep_summary_rapids_avgdurationinbedmain:14dhist,f_slp:fitbit_sleep_summary_rapids_countepisodemain:14dhist,f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:14dhist,f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:14dhist,f_slp:fitbit_sleep_summary_rapids_firstwaketimemain:14dhist,f_slp:fitbit_sleep_summary_rapids_lastwaketimemain:14dhist,f_slp:fitbit_sleep_intraday_rapids_avgdurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_avgdurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_maxdurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_maxdurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_sumdurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_sumdurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_countepisodeasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_countepisodeawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_stddurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_stddurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mindurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mindurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mediandurationasleepunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_mediandurationawakeunifiedmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_ratiocountasleepunifiedwithinmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_ratiocountawakeunifiedwithinmain:14dhist,f_slp:fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain:14dhist,…,f_loc:phone_locations_barnett_probpause_norm:7dhist,f_loc:phone_locations_barnett_rog_norm:7dhist,f_loc:phone_locations_barnett_siglocentropy_norm:7dhist,f_loc:phone_locations_barnett_siglocsvisited_norm:7dhist,f_loc:phone_locations_barnett_stdflightdur_norm:7dhist,f_loc:phone_locations_barnett_stdflightlen_norm:7dhist,f_loc:phone_locations_barnett_wkenddayrtn_norm:7dhist,f_loc:phone_locations_doryab_avglengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_avgspeed_norm:7dhist,f_loc:phone_locations_doryab_homelabel_norm:7dhist,f_loc:phone_locations_doryab_locationentropy_norm:7dhist,f_loc:phone_locations_doryab_locationvariance_norm:7dhist,f_loc:phone_locations_doryab_loglocationvariance_norm:7dhist,f_loc:phone_locations_doryab_maxlengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_minlengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_movingtostaticratio_norm:7dhist,f_loc:phone_locations_doryab_normalizedlocationentropy_norm:7dhist,f_loc:phone_locations_doryab_numberlocationtransitions_norm:7dhist,f_loc:phone_locations_doryab_numberofsignificantplaces_norm:7dhist,f_loc:phone_locations_doryab_outlierstimepercent_norm:7dhist,f_loc:phone_locations_doryab_radiusgyration_norm:7dhist,f_loc:phone_locations_doryab_stdlengthstayatclusters_norm:7dhist,f_loc:phone_locations_doryab_timeathome_norm:7dhist,f_loc:phone_locations_doryab_timeattop1location_norm:7dhist,f_loc:phone_locations_doryab_timeattop2location_norm:7dhist,f_loc:phone_locations_doryab_timeattop3location_norm:7dhist,f_loc:phone_locations_doryab_totaldistance_norm:7dhist,f_loc:phone_locations_doryab_varspeed_norm:7dhist,f_loc:phone_locations_locma

In [13]:
target = rapids_weekly.select("dep_weeklysubscale_endterm_merged_true").to_numpy().tolist()
features = rapids_weekly.to_numpy().tolist()

In [ ]:
sleep_1 = preprocessor_INS_W1.import_csv_feature_data("sleep")
wifi_1 = preprocessor_INS_W1.import_csv_feature_data("wifi")
bluetooth_1 = preprocessor_INS_W1.import_csv_feature_data("bluetooth")
call_1 = preprocessor_INS_W1.import_csv_feature_data("call")
location_1 = preprocessor_INS_W1.import_csv_feature_data("location")
screen_1 = preprocessor_INS_W1.import_csv_feature_data("screen")
steps_1 = preprocessor_INS_W1.import_csv_feature_data("steps")

In [120]:
sleep_wifi_temp_1 = preprocessor_INS_W1.merge_survey_to_feature(sleep_1, wifi_1, join_type="inner")
bluetooth_call_temp_1 = preprocessor_INS_W1.merge_survey_to_feature(bluetooth_1, call_1, join_type="inner")
location_screen_temp_1 = preprocessor_INS_W1.merge_survey_to_feature(location_1, screen_1, join_type="inner")

sleep_wifi_steps_1 = preprocessor_INS_W1.merge_survey_to_feature(sleep_wifi_temp_1, steps_1, join_type="inner")
bluetooth_call_location_screen = preprocessor_INS_W1.merge_survey_to_feature(bluetooth_call_temp_1, location_screen_temp_1, join_type="inner")
merged_all_temp = preprocessor_INS_W1.merge_survey_to_feature(sleep_wifi_steps_1, bluetooth_call_location_screen, join_type="inner")